In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [6]:
df = pd.read_csv('pokemon.csv')

In [7]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [8]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]


In [10]:
df.dtypes

isLegendary      int32
Generation       int64
Type_1          object
Type_2          object
HP               int64
Attack           int64
Defense          int64
Sp_Atk           int64
Sp_Def           int64
Speed            int64
Color           object
Egg_Group_1     object
Height_m       float64
Weight_kg      float64
Body_Style      object
dtype: object

In [9]:
df['isLegendary'] = df['isLegendary'].astype(int)

In [11]:
def dummy_creation(df, dummy_categories):
    for i in dummy_categories:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df,df_dummy],axis=1)
        df = df.drop(i, axis=1)
    return(df)

In [12]:
df = dummy_creation(df, ['Egg_Group_1', 'Body_Style', 'Color','Type_1', 'Type_2'])

In [13]:
def train_test_splitter(DataFrame, column):
    df_train = DataFrame.loc[df[column] != 1]
    df_test = DataFrame.loc[df[column] == 1]

    df_train = df_train.drop(column, axis=1)
    df_test = df_test.drop(column, axis=1)

    return(df_train, df_test)

df_train, df_test = train_test_splitter(df, 'Generation')

In [14]:
def label_delineator(df_train, df_test, label):
    
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    test_data = df_test.drop(label,axis=1).values
    test_labels = df_test[label].values
    return(train_data, train_labels, test_data, test_labels)

In [15]:
train_data, train_labels, test_data, test_labels = label_delineator(df_train, df_test, 'isLegendary')

In [16]:
def data_normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return(train_data, test_data)

train_data, test_data = data_normalizer(train_data, test_data)

In [17]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation='relu', input_shape=[length,]))
model.add(keras.layers.Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
570/570 [==============================] - 3s 5ms/sample - loss: 0.5122 - acc: 0.9228
Epoch 2/400
570/570 [==============================] - 0s 70us/sample - loss: 0.4076 - acc: 0.9263
Epoch 3/400
570/570 [==============================] - 0s 56us/sample - loss: 0.3517 - acc: 0.9263
Epoch 4/400
570/570 [==============================] - 0s 56us/sample - loss: 0.3194 - acc: 0.9263
Epoch 5/400
570/570 [==============================] - 0s 56us/sample - loss: 0.2983 - acc: 0.9263
Epoch 6/400
570/570 [==============================] - 0s 56us/sample - loss: 0.2842 - acc: 0.9263
Epoch 7/400
570/570 [==============================] - 0s 56us/sample - loss: 0.2737 - acc: 0.9263
Epoch 8/400
570/570 [==============================] - 0s 56us/sample - loss: 0.2655 - acc: 0.9263
Epoch 9/400
570/570 [==============================] - 0s 56us/sample - loss: 0.2588 - acc: 0.9263
Epoch 10/400
570/570 [==============================] - 0s 71us/sample - loss: 0.2528 - acc: 0.9263
Epoch 11/4

In [20]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f'Our test accuracy was {accuracy_value}')

151/151 [==============================] - 0s 212us/sample - loss: 0.0344 - acc: 0.9801
Our test accuracy was 0.9801324605941772


In [21]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)

In [22]:
predictor(test_data, test_labels, 149)

This was correctly predicted to be a "1"!
